# Obtenção e primeira exploração de dados

In [ ]:
import pandas as pd
from tqdm import tqdm

import database
from voos import voos_url

In [ ]:
microdados = pd.DataFrame()

urls = [ ]
for year in range( 2020, 2020 + 1 ):
	for month in range( 1, 12 + 1 ):
		month = str( month ) if month >= 10 else "0" + str( month )
		url = f"https://www.gov.br/anac/pt-br/assuntos/regulados/empresas-aereas/envio-de-informacoes/microdados/basica{year}-{month}.zip"
		urls.append( url )

for url in tqdm( urls ):
	parcial = pd.read_csv( url, encoding = "latin1", sep = ";" ).sample( frac = 0.05, random_state = 2 )
	microdados = pd.concat( [ microdados, parcial ], axis = 0 )

url_est = 'https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/dados-estatisticos/arquivos/resumo_anual_2010.csv'


In [ ]:
# Da definição:

# Etapa Básica (flight stage): As etapas básicas são aquelas realizadas pela aeronave desde a sua decolagem até o próximo pouso, independentemente de onde tenha ocorrido o embarque ou o desembarque do objeto de transporte.
#
# Os dados estatísticos das etapas básicas representam o status da aeronave em cada etapa do voo, apresentando a movimentação de cargas e passageiros entre os aeródromos de origem e destino da aeronave. É a operação de uma aeronave entre um a decolagem e o próximo pouso, a ligação entre dois aeródromos.

# Etapa Combinada (On flight origin and destination - OFOD): As etapas combinadas identificam os pares de origem, onde ocorreu o embarque do objeto de transporte, e destino, onde ocorreu o desembarque do objeto de transporte, independentemente da existência de aeródromos intermediários, atendidos por determinado voo.

# É a etapa de voo vista com foco no objeto de transporte (pessoas e/ou cargas), com base no embarque e no desembarque nos aeródromos relacionados. Os dados estatísticos da etapa combinada informam a origem e destino dos passageiros e cargas transportadas no voo, independentemente das escalas realizadas.


#Aqui olharemos primeiramente para a etapa basica.

In [ ]:
microdados.duplicated( [ "id_basica" ] ).sum()
# de Fato, id basica é uma pk ( identificador unico ) nessa tabela, e não temos nenhum registro duplicado!

In [ ]:
(microdados.isna().sum( axis = 0 ) / microdados.shape[ 0 ]).sort_values( ascending = False )

# nr escala destino é 100% nulo. Doideira ( Isso provavelmente é algum tipo de informação que foi dropada ao longo do tempo. )

In [ ]:

unique = microdados.nm_empresa.value_counts()
cumulative = unique.cumsum()
cumulative = cumulative / unique.sum()
cumulative.sort_values( ascending = True ).iloc[ :15 ].plot( kind = "bar" )

# Nenhuma surpresa aqui: a gigantesca maioria das etapas é realizada por poucas empresas
# Aprox. 80% das etapas é feita por apenas 3;
#

In [ ]:
microdados.groupby( [ "nr_mes_referencia", "nm_mes_referencia" ] ).count()[ "id_basica" ].sort_index( ascending = True ).plot( kind = "bar" )

In [ ]:
microdados[ "nr_dia_semana_referencia" ] = pd.to_datetime( microdados[ "dt_referencia" ], infer_datetime_format = True ).dt.dayofweek

microdados.groupby( [ "nr_dia_semana_referencia", "nm_dia_semana_referencia" ] ).count()[ "id_basica" ].sort_index( ascending = True ).plot( kind = "bar" )

# Aqui temos uma surpresa: Fim de semanas são dias com menos voos? Interessante. [quinta, sexta, segunda, quarta, terca ] são dias de ocupação bem proxima.

In [ ]:
microdados.query( "nm_pais != 'BRASIL'" ).groupby( "nm_pais" ).nunique()[ "nm_empresa" ].nlargest( 10 ).plot( kind = "bar" )
# Essas são as nacionalidades das empresas não brasileiras dentro do dataset. USA domina, seguido de argentina, chile, venezuela

In [ ]:
microdados.groupby( [ "nm_municipio_destino", "nm_municipio_origem", "nm_empresa" ] ).nunique()

In [ ]:
microdados[ "nr_voo" ].value_counts()
# O numero de voo mais comum é 2317 , seguido de 2316, seguido de 248, 15 e 247
# Os menos comuns tem apenas 1 e são : 2263 , 6565, 595, 6566 , 593
# Atente-se à amostragem

In [ ]:
microdados.query( "nr_voo == 2317" )[ [ "nm_empresa", "nr_voo", "nr_singular", "id_di", "nm_municipio_destino", "nm_municipio_origem", "dt_referencia" ] ]
# O que identifica uma rota ( ORIGEM - DESTINO )  de maneira unica?!

In [ ]:
microdados[ "dt_referencia" ] = pd.to_datetime( microdados.dt_referencia, infer_datetime_format = True )

In [ ]:
microdados.groupby( [ "nr_ano_referencia", "nr_mes_referencia" ] ).mean()[ [ "nr_ask",
                                                                             "nr_rpk",
                                                                             "nr_atk",
                                                                             "nr_rtk",
                                                                             "kg_peso" ] ].sort_index( ascending = True ).plot()

# Análise dos Atrasos

In [ ]:
urls = list( voos_url )[ :5 ]


def download_voos( url ):
	return pd.read_csv( url, skiprows = 1, sep = ";" )


voos = pd.DataFrame()
for url in tqdm( urls ):
	parcial = download_voos( url )

	voos = pd.concat( (parcial, voos), axis = 0 )

In [ ]:
for data in [ "Chegada Prevista", "Chegada Real", "Partida Prevista", "Partida Real" ]:
	voos[ data ] = pd.to_datetime( voos[ data ], infer_datetime_format = True )

In [ ]:
voos[ "ano" ] = voos[ "Partida Real" ].dt.year
voos[ "mes" ] = voos[ "Partida Real" ].dt.month
voos[ "semana_ano" ] = voos[ "Partida Real" ].dt.week
voos[ "dia_semana" ] = voos[ "Partida Real" ].dt.dayofweek
voos[ "atraso_chegada" ] = (voos[ "Chegada Real" ] - voos[ "Chegada Prevista" ]).dt.seconds / 60
voos[ "atraso_saida" ] = (voos[ "Partida Real" ] - voos[ "Partida Prevista" ]).dt.seconds / 60
voos[ "atraso_ajustado" ] = voos[ "atraso_chegada" ] - voos[ "atraso_saida" ]
voos[ "atraso_chegada_abs" ] = abs( voos[ "atraso_chegada" ] )
voos[ "atraso_saida_abs" ] = abs( voos[ "atraso_saida" ] )
voos[ "atraso_ajustado_abs" ] = abs( voos[ "atraso_ajustado" ] )

In [ ]:
voos[ "atraso_chegada" ].plot( kind = "hist", bins = 100 )

In [ ]:
voos[ "duracao_real" ] = (voos[ "Chegada Real" ] - voos[ "Partida Real" ]).dt.seconds / 60
voos[ "duracao_prevista" ] = (voos[ "Chegada Prevista" ] - voos[ "Partida Prevista" ]).dt.seconds / 60

In [ ]:
(voos.query( "atraso_chegada > 0 & atraso_chegada < 240" ).groupby( [ "ano", "mes" ] )
 .agg(
	percentil_10 = ("atraso_chegada", lambda x: x.quantile( 0.10 )),
	percentil_20 = ("atraso_chegada", lambda x: x.quantile( 0.20 )),
	percentil_30 = ("atraso_chegada", lambda x: x.quantile( 0.30 )),
	percentil_40 = ("atraso_chegada", lambda x: x.quantile( 0.40 )),
	percentil_50 = ("atraso_chegada", lambda x: x.quantile( 0.50 )),
	percentil_60 = ("atraso_chegada", lambda x: x.quantile( 0.60 )),
	percentil_70 = ("atraso_chegada", lambda x: x.quantile( 0.70 )),
	percentil_80 = ("atraso_chegada", lambda x: x.quantile( 0.80 )),
	percentil_90 = ("atraso_chegada", lambda x: x.quantile( 0.90 )),
)[ [ "percentil_10",
	 "percentil_20",
	 "percentil_30",
	 "percentil_40",
	 "percentil_50",
	 "percentil_60",
	 "percentil_70",
	 "percentil_80",
	 "percentil_90" ] ].sort_index( ascending = True ).plot( figsize = (20, 20) )
 )
# Doideira : um salto gigantesco no tamanho dos atrasos_saida e chegada, mas ambos acompanham uns aos outros.
# #Uma possivel explicação é a presença da olimpiada no mesmo ano /periodo em que vemos esse crescimento acentuado. No entanto, era de se esperar que passado esse evento, tudo se normalizasse, mas
# não é o que observamos.

In [ ]:
voos.query( "atraso_chegada > 0" ).groupby( [ "ano", "mes" ] )[ [ "atraso_chegada",
                                                                  "atraso_saida" ] ].mean().sort_index( ascending = True ).plot()

# Analisando somente os voos que atrasam, Vemos que a duração desses atrasos permanece razoavelmente constante. ao redor de 600 +- 100, Com excessão desse evento proximo a 2017-1

In [ ]:
# Sera que os atrasos seguem algum tipo de padrão sazonal?

voos.groupby( "semana_ano" ).mean()[ [ "atraso_chegada",
                                       "atraso_saida" ] ].plot()

# Temos um descolamento entre  chegada e saida ao redor da semana 40.

# Nao estou vendo muito padrão nao. Talvez necessitamos de mais variaveis, ou talvez seja com o dia do mes / dia da semana.
# Talvez durante a transição de horario de verão?

In [ ]:
voos.query( "atraso_chegada > 0 " ).groupby( "dia_semana" ).median()[ [ "atraso_chegada",
                                                                        "atraso_saida" ] ].plot()

# Parece haver um efeito sazonal do dia da semana no que diz respeito aos atrasos.
# Peak atrasos ~ (terca )
# Menor ~ ( quinta)

In [ ]:
voos

In [ ]:
ids_linhas = [ 'N', 'E', 'R' ]
ids_dis = [ '0', '4', 'C' ]

voos = voos.rename( {
		'Código Tipo Linha'      : 'id_tipo_linha',
		'Código Autorização (DI)': 'cd_di'
		}, axis = 1 )
voos = voos.query( 'id_tipo_linha in @ids_linhas' )
voos = voos.query( 'cd_di in @ids_dis' )
# Filtrando apenas pelos municípios de SP e RJ, vemos que existem algumas empresas que nem conhecemos, bem como aeroportos que não conhecemos.

# Um filtro que faremos daqui em diante vai ser filtrar apenas voos de passageiros.
# Isso signfica apenas os códigos ( N , I, E , R).

# Um segundo filtro diz respeito aos voos internacionais: Não consideramos eles.
# Isso inviabiliza os voos com código I.

# Portanto consideramos apenas os códigos ( N , E , R )

In [ ]:
# Outra dimensão a ser considerada é o Digito Identificador.
# Ele diz respeito ao tipo de voo realizado. Podem ser dos seguintes tipos:
# Regular, Improdutivo, Não regular.
# Nesta analise consideraremos apenas os voos regulares.
# Estes são sinalizados com cd_di pertencente à ( 0 , 4 , C)

In [ ]:
# Outro ponto a ser considerado é a situação do voo.
# Consideraremos apenas os voos Realizados

In [ ]:
# Correlação de atrasos com outras features
# Algum aeroporto atrasa mais que os outros? Isso pode ser causado pelo clima do aeroporto ou pela movimentação daquele aeroporto
# Horario do dia. ( Período manhã / tarde / noite )
# Sera que algum voo atrasa mais?
# Alguma companhia aerea?

# Lotação?

# Preparação dos dados para o modelo de Machine Learning
## Limpeza
## Tratamento
## Feature engineering
## Enriquecimento da base

In [ ]:
voos = pd.read_sql( '''
with
	resumo_voos as (
	select
		date(dt_partida_prevista)  as dt_referencia
	  , dt_chegada_real
	  , dt_chegada_prevista
	  , sg_empresa_icao
	  , origem.nm_municipio  as municipio_origem
	  , destino.nm_municipio as municipio_destino
	  , timestampdiff( minute ,    dt_chegada_prevista , dt_chegada_real ) as atrasado
		from
			voos_sp_sj               voos
			inner join nrm_empresa   empresa on empresa.id_empresa = voos.id_empresa
			inner join nrm_aeroporto origem on origem.id_aerodromo = voos.id_aerodromo_origem
			inner join nrm_aeroporto destino on destino.id_aerodromo = voos.id_aerodromo_destino
		where
				dt_partida_prevista >= '2014-12-30 00:00:00'
		order by
			dt_partida_prevista
	)

SELECT *
	FROM
		resumo_voos
''', database.engine )
clima = pd.read_sql( """
select
	nm_municipio
  , dt_referencia + interval 1 day as dt_referencia
  , precipitacao_mm
  , pressao_mb
  , umidade_relativa
  , temperatura_k
  , vento_ms
	from
		resumo_clima
                    """, database.engine )

# Modelo Final

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_roc_curve, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from category_encoders import TargetEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
voos = pd.read_sql( '''
with
	resumo_voos as (
	select
		date(dt_partida_prevista)  as dt_referencia
	  , dt_chegada_real
	  , dt_chegada_prevista
	  , sg_empresa_icao
	  , origem.nm_municipio  as municipio_origem
	  , destino.nm_municipio as municipio_destino
	  , timestampdiff( minute ,    dt_chegada_prevista , dt_chegada_real ) as atrasado
		from
			voos_sp_sj               voos
			inner join nrm_empresa   empresa on empresa.id_empresa = voos.id_empresa
			inner join nrm_aeroporto origem on origem.id_aerodromo = voos.id_aerodromo_origem
			inner join nrm_aeroporto destino on destino.id_aerodromo = voos.id_aerodromo_destino
		where
				dt_partida_prevista >= '2014-12-30 00:00:00'
		order by
			dt_partida_prevista
	)

SELECT *
	FROM
		resumo_voos
''', database.engine )
clima = pd.read_sql( """
select
	nm_municipio
  , dt_referencia + interval 1 day as dt_referencia
  , precipitacao_mm
  , pressao_mb
  , umidade_relativa
  , temperatura_k
  , vento_ms
	from
		resumo_clima
                    """, database.engine )
for column in voos.columns:
	if column.startswith( "dt" ):
		voos[ column ] = pd.to_datetime( voos[ column ], infer_datetime_format = True )
for column in clima.columns:
	if column.startswith( "dt" ):
		clima[ column ] = pd.to_datetime( clima[ column ], infer_datetime_format = True )
voos = voos.query( "atrasado > -60" )
voos[ "atrasado" ] = voos[ "atrasado" ] >= 20

In [ ]:
df = voos.merge( clima, how = "left",
		left_on = [ "municipio_origem", "dt_referencia" ],
		right_on = [ "nm_municipio", "dt_referencia" ] )
df[ "dia_semana" ] = df[ "dt_referencia" ].dt.dayofweek
df[ "semana_ano" ] = df[ "dt_referencia" ].dt.weekofyear
numericas = [ "precipitacao_mm", "pressao_mb", "umidade_relativa", "temperatura_k", "vento_ms" ]
categoricas = [ "sg_empresa_icao",
                "municipio_origem",
                "municipio_destino",
                "dia_semana",
                "semana_ano" ]
columns_keep = [ ]
columns_keep.extend( numericas )
columns_keep.extend( categoricas )
columns_keep.append( "atrasado" )
columns_keep.append( "dt_referencia" )
df = df[ columns_keep ]
df = (df
      .sort_values( "dt_referencia" )
      .groupby( "municipio_origem", as_index = False )
      .fillna( method = "ffill" )
      .dropna())
df

In [ ]:

df = (df
	  .sort_values( "dt_referencia" )
	  .groupby( "municipio_origem", as_index = False )
	  .fillna( method = "ffill" )
	  .dropna())
df

In [ ]:
df_y = df[ [ "atrasado" ] ]
df_x = df[ columns_keep ].drop( [ "atrasado", "dt_referencia" ], axis = 1 )
(df_train_x, df_test_x,
 df_train_y, df_test_y) = train_test_split( df_x, df_y,
		train_size = 0.8,
		stratify = df_y )
encoder = TargetEncoder( cols = categoricas,
		handle_unknown = 'value' ).fit( df_train_x, df_train_y )
df_train_x = encoder.transform( df_train_x )
df_test_x = encoder.transform( df_test_x )

In [ ]:
forest: RandomForestClassifier = RandomForestClassifier( max_depth = 4, random_state = 1, )
forest: RandomForestClassifier = forest.fit( df_train_x, df_train_y )
df_train_yhat_forest = forest.predict_proba( df_train_x )[ :, [ 1 ] ]
df_test_yhat_forest = forest.predict_proba( df_test_x )[ :, [ 1 ] ]
logistic: LogisticRegression = LogisticRegression()
logistic = logistic.fit( df_train_x, df_train_y )
df_train_yhat_logistic = logistic.predict_proba( df_train_x )[ :, [ 1 ] ]
df_test_yhat_logistic = logistic.predict_proba( df_test_x )[ :, [ 1 ] ]

In [ ]:

for train, test in ((df_train_yhat_forest, df_test_yhat_forest),
                    (df_train_yhat_logistic, df_test_yhat_logistic)):
	threshold = np.percentile( train, 80 )
	print( threshold )
	y_true = df_test_y
	y_pred = test >= threshold
	normalize = None
	cm = confusion_matrix( y_true,
			y_pred, normalize = normalize,
			)
	predictions = ConfusionMatrixDisplay.from_predictions( y_true = y_true, y_pred = y_pred,
			normalize = normalize, cmap = 'Blues' )
	plt.show()


In [ ]:
fig, ax = plt.subplots()
ax.plot( [ 0, 1 ], [ 0, 1 ] )
plot_roc_curve( forest, df_train_x, df_train_y, ax = ax )
plot_roc_curve( logistic, df_train_x, df_train_y, ax = ax )